##Bi-directional LSTM for sentiment analysis using IMDB dataset 

Preparing data

In [ ]:
import torch #importing pytorch
from torchtext import data #pytorch library for preprocessing
import torch.nn.functional as F
#setting random seed
SEED = 1234
# setting manual seed using our random seed to get the same random number
torch.manual_seed(SEED)
# running on the CuDNN backend
torch.backends.cudnn.deterministic = True
# tokenizing text using spacy tokenizer and include lengths for packed padded sequences
TEXT = data.Field(tokenize = 'spacy',include_lengths=True) 
# Labelling our dataset and setting tensor to FloatTensors A label field is a shallow wrapper around a standard field designed to hold labels for a classification task
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
from torchtext import datasets # to get torch dataset
# downloading the IMDb dataset and spliting it into the canonical train/test splits as torchtext.datasets objects
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [ ]:
# Checking length of train test splits
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [ ]:
train_data[0].text

['Sydney',
 'Lumet',
 'has',
 "n't",
 'had',
 'a',
 'box',
 'office',
 'hit',
 'in',
 '20',
 'years',
 'and',
 'yet',
 'at',
 '83',
 'has',
 'managed',
 'to',
 'churn',
 'out',
 'a',
 'tight',
 ',',
 'well',
 '-',
 'cast',
 ',',
 'suspenseful',
 'thriller',
 'set',
 'in',
 'his',
 'old',
 'stamping',
 'ground',
 ',',
 'New',
 'York',
 'City',
 '.',
 '(',
 'How',
 'he',
 'got',
 'insurance',
 ',',
 'let',
 'alone',
 'the',
 'budget',
 'after',
 'all',
 'those',
 'flops',
 ',',
 'is',
 'a',
 'mystery',
 'also',
 ')',
 '.',
 'The',
 'story',
 'is',
 'a',
 'pretty',
 'grim',
 'one',
 'and',
 'the',
 'characters',
 'are',
 'not',
 'particularly',
 'likable',
 'but',
 'it',
 'held',
 'me',
 'on',
 'the',
 'edge',
 'of',
 'my',
 'seat',
 'till',
 'the',
 'final',
 'scene.<br',
 '/><br',
 '/>Two',
 'brothers',
 'with',
 'pressing',
 'financial',
 'problems',
 'conspire',
 'to',
 'rob',
 'a',
 'suburban',
 'jewelry',
 'store',
 'owned',
 'by',
 'their',
 'elderly',
 'parents',
 '.',
 'The',
 'o

In [ ]:
train_data[0].label

'pos'

In [ ]:
print(vars(train_data.examples[0]))

{'text': ['Sydney', 'Lumet', 'has', "n't", 'had', 'a', 'box', 'office', 'hit', 'in', '20', 'years', 'and', 'yet', 'at', '83', 'has', 'managed', 'to', 'churn', 'out', 'a', 'tight', ',', 'well', '-', 'cast', ',', 'suspenseful', 'thriller', 'set', 'in', 'his', 'old', 'stamping', 'ground', ',', 'New', 'York', 'City', '.', '(', 'How', 'he', 'got', 'insurance', ',', 'let', 'alone', 'the', 'budget', 'after', 'all', 'those', 'flops', ',', 'is', 'a', 'mystery', 'also', ')', '.', 'The', 'story', 'is', 'a', 'pretty', 'grim', 'one', 'and', 'the', 'characters', 'are', 'not', 'particularly', 'likable', 'but', 'it', 'held', 'me', 'on', 'the', 'edge', 'of', 'my', 'seat', 'till', 'the', 'final', 'scene.<br', '/><br', '/>Two', 'brothers', 'with', 'pressing', 'financial', 'problems', 'conspire', 'to', 'rob', 'a', 'suburban', 'jewelry', 'store', 'owned', 'by', 'their', 'elderly', 'parents', '.', 'The', 'only', 'victim', 'is', 'going', 'to', 'be', 'the', 'insurance', 'company', '.', 'The', 'robbery', 'goes

In [ ]:
import random
#creating valdation set and setting random_state so that we get training and validation set same each time
train_data, valid_data = train_data.split(random_state = random.seed(SEED),split_ratio=0.8)

In [ ]:
# Checking length of train,test, validation splits
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 20000
Number of validation examples: 5000
Number of testing examples: 25000


In [ ]:
# building the vocabulary, only keeping the most common max_size tokens using our training set
MAX_VOCAB_SIZE = 20000 

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [ ]:
# Unique tokens in TEXT vocabulary and LABEL vocabulary
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 20002
Unique tokens in LABEL vocabulary: 2


In [ ]:
#to view the most common words in the vocabulary and their frequencies
print(TEXT.vocab.freqs.most_common(20))

[('the', 231896), (',', 220268), ('.', 189990), ('and', 125248), ('a', 125148), ('of', 115090), ('to', 107050), ('is', 87499), ('in', 70003), ('I', 62195), ('it', 61349), ('that', 56156), ('"', 50662), ("'s", 49440), ('this', 48213), ('-', 42289), ('/><br', 40628), ('was', 40052), ('as', 35049), ('with', 34036)]


In [ ]:
# We can also see the vocabulary directly using either the stoi (string to int) or itos (int to string) method.
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [ ]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f8eefc152f0>, {'neg': 0, 'pos': 1})


In [ ]:
# We then create the iterators. We iterate over these in the training/evaluation loop, and 
#they return a batch of examples (indexed and converted into tensors) at each iteration.
#We'll use a BucketIterator which is a special type of iterator that will return
#a batch of examples where each example is of a similar length, minimizing 
#the amount of padding per example.
#We also want to place the tensors returned by the iterator on the 
#GPU (if you're using one). PyTorch handles this using torch.device, 
#we then pass this device to the iterator.

BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,sort_within_batch=True, #for packed_padded_sequences
    device = device)

In [ ]:
#testing the iterators 
#number of rows depends on the longest document in the respective batch
print('Train')
for batch in train_iterator:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break
    
print('\nValid:')
for batch in valid_iterator:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break
    
print('\nTest:')
for batch in test_iterator:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break

Train
Text matrix size: torch.Size([509, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([60, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([42, 128])
Target vector size: torch.Size([128])


Building our model

In [ ]:
import torch.nn as nn # neural network library of pytorch
# Defining our RNN class as a sub-class of nn.Module 
class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=NUM_LAYERS,
                           bidirectional=BIDIRECTIONAL)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        
    def forward(self, text, text_length):

        #[sentence len, batch size] => [sentence len, batch size, embedding size]
        embedded = self.embedding(text)
        
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_length)
        
        packed_output, (hidden, cell) = self.rnn(packed)
        
        # combine both directions
        combined = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        
        return self.fc(combined.squeeze(0)).view(-1)    

In [ ]:
# to create an instance of our RNN class by passing INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM and OUTPUT_DIM
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 128
HIDDEN_DIM = 128
NUM_LAYERS = 2
OUTPUT_DIM = 1
BIDIRECTIONAL=True

model = LSTM(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM)

In [ ]:
def count_parameters(model):
    """tell us how many trainable parameters our model has so we can compare the number of parameters across different models"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,219,969 trainable parameters


In [ ]:
print(model)

LSTM(
  (embedding): Embedding(20002, 128)
  (rnn): LSTM(128, 128, num_layers=2, bidirectional=True)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)


Training our model

In [ ]:

import torch.optim as optim #optimizer
# we use ADAM with learning rate as 0.0001
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
#putting model and criterion to our device
model = model.to(device)

In [ ]:
def compute_binary_accuracy(model, data_iterator, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(data_iterator):
            text, text_lengths = batch_data.text
            logits = model(text, text_lengths)
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            num_examples += batch_data.label.size(0)
            correct_pred += (predicted_labels == batch_data.label.long()).sum()
        return correct_pred.float()/num_examples * 100

In [ ]:
import time 
start_time = time.time()
NUM_EPOCHS=15
for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_iterator):
        text,text_lengths = batch_data.text
        ### FORWARD AND BACK PROP
        logits = model(text,text_lengths)
        cost = F.binary_cross_entropy_with_logits(logits, batch_data.label)
        #setting gradients to zero
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_iterator):03d} | '
                   f'Cost: {cost:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_binary_accuracy(model, train_iterator, device):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_binary_accuracy(model, valid_iterator, device):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')

Epoch: 001/015 | Batch 000/157 | Cost: 0.6904
Epoch: 001/015 | Batch 050/157 | Cost: 0.6881
Epoch: 001/015 | Batch 100/157 | Cost: 0.6871
Epoch: 001/015 | Batch 150/157 | Cost: 0.6871
training accuracy: 59.45%
valid accuracy: 59.04%
Time elapsed: 0.30 min
Epoch: 002/015 | Batch 000/157 | Cost: 0.6632
Epoch: 002/015 | Batch 050/157 | Cost: 0.6572
Epoch: 002/015 | Batch 100/157 | Cost: 0.6055
Epoch: 002/015 | Batch 150/157 | Cost: 0.5178
training accuracy: 73.04%
valid accuracy: 72.52%
Time elapsed: 0.59 min
Epoch: 003/015 | Batch 000/157 | Cost: 0.5450
Epoch: 003/015 | Batch 050/157 | Cost: 0.5334
Epoch: 003/015 | Batch 100/157 | Cost: 0.4344
Epoch: 003/015 | Batch 150/157 | Cost: 0.5122
training accuracy: 77.92%
valid accuracy: 75.84%
Time elapsed: 0.89 min
Epoch: 004/015 | Batch 000/157 | Cost: 0.4082
Epoch: 004/015 | Batch 050/157 | Cost: 0.4937
Epoch: 004/015 | Batch 100/157 | Cost: 0.4615
Epoch: 004/015 | Batch 150/157 | Cost: 0.4379
training accuracy: 81.31%
valid accuracy: 78.82%

In [ ]:
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
#prints the confusion matrix as well as test = True
print(f'Test accuracy: {compute_binary_accuracy(model, test_iterator, device):.2f}%')

Total Training Time: 7.50 min
Test accuracy: 85.64%


In [ ]:
import spacy 
nlp = spacy.load('en')
def predict_sentiment(model, sentence):
    """ inputs the sentence and the model and outputs the sentiment of sentence based on the model"""
    model.eval()
    #creating list of tokens for our sentence
    tokenized = [token.text for token in nlp.tokenizer(sentence)]
    # index of token in TEXT.vocab
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor,length_tensor))
    if (prediction.item())>=0.5:
        return "Positive with score "+ str(prediction.item())
    else:
        return "Negative with score "+ str(prediction.item())
        

In [ ]:
predict_sentiment(model, "I really love this movie.This movie is so great!")

'Positive with score 0.9901903867721558'